# Autoregressive (자동회귀) 문장 생성 - HyperClova 모델

HyperClova 모델을 사용한 자동 회귀적인 텍스트 생성 예제입니다.
HyperClova는 네이버에서 개발한 한국어 특화 대화형 언어모델로,
채팅 템플릿을 사용하여 자연스러운 대화형 텍스트를 생성합니다.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# HyperClova 모델 및 토크나이저 초기화
# HyperClova는 네이버에서 개발한 한국어 특화 대화형 언어모델
model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 문장 시작 부분 (한국어로 변경)
input_text = "옛날 옛적에"
print(f"원본 텍스트: {input_text}")

# 토크나이저 정보 확인
print(f"토크나이저 vocab 크기: {tokenizer.vocab_size}")
print(f"특수 토큰들: {tokenizer.special_tokens_map}")

# 간단한 토큰화 테스트
input_ids = tokenizer.encode(input_text, return_tensors="pt")
print(f"토큰화된 결과: {input_ids}")
print(f"토큰 개수: {input_ids.shape[1]}")
input_ids

원본 텍스트: 옛날 옛적에
토크나이저 vocab 크기: 110491
특수 토큰들: {'bos_token': '<|endoftext|>', 'eos_token': '<|endofturn|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|fim_prefix|>', '<|fim_middle|>', '<|fim_suffix|>', '<|endofprompt|>', '<|_unuse_missing_100256|>', '<|_unuse_missing_100261|>', '<|_unuse_missing_100262|>', '<|_unuse_missing_100263|>', '<|_unuse_missing_100264|>', '<|_unuse_missing_100265|>', '<|_unuse_missing_100266|>', '<|_unuse_missing_100267|>', '<|_unuse_missing_100268|>', '<|_unuse_missing_100269|>', '<|_unuse_missing_100270|>', '<|_unuse_missing_100271|>', '<|im_start|>', '<|im_end|>', '<|stop|>', '<|endofturn|>', '<repo_name>', '<file_sep>', '<issue_start>', '<issue_comment>', '<issue_closed>', '<jupyter_start>', '<jupyter_text>', '<jupyter_code>', '<jupyter_output>', '<jupyter_script>', '<empty_output>', '<code_to_intermediate>', '<intermediate_to_code>', '<pr>', '<pr_status>', '<pr_is_merged>', '<pr_base>', '<pr

tensor([[ 36092,    249, 104013, 102476,  82068,  19954]])

In [18]:
# HyperClova 채팅 템플릿을 사용한 문장 생성
# HyperClova는 채팅 형식의 템플릿을 사용하여 대화형 텍스트 생성

# 채팅 형식으로 입력 구성
chat = [
    {"role": "system", "content": "당신은 창의적인 이야기 작가입니다. 주어진 프롬프트로 흥미로운 이야기를 써주세요."},
    {"role": "user", "content": f"{input_text}로 시작하는 이야기를 써주세요."}
]

print("=== 채팅 템플릿 구성 ===")
for i, message in enumerate(chat):
    print(f"{i+1}. {message['role']}: {message['content']}")

# 채팅 템플릿 적용
inputs = tokenizer.apply_chat_template(
    chat,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
)

print(f"\n=== 템플릿 적용 결과 ===")
print(f"입력 키들: {list(inputs.keys())}")
print(f"input_ids 형태: {inputs['input_ids'].shape}")

# GPU 사용 가능하면 GPU로 이동
if torch.cuda.is_available():
    inputs = {k: v.to("cuda") for k, v in inputs.items()}
    print("GPU로 이동 완료")
else:
    print("CPU 사용 중")

=== 채팅 템플릿 구성 ===
1. system: 당신은 창의적인 이야기 작가입니다. 주어진 프롬프트로 흥미로운 이야기를 써주세요.
2. user: 옛날 옛적에로 시작하는 이야기를 써주세요.

=== 템플릿 적용 결과 ===
입력 키들: ['input_ids', 'attention_mask']
input_ids 형태: torch.Size([1, 44])
GPU로 이동 완료


In [19]:
# HyperClova 텍스트 생성
# generate 메서드의 주요 파라미터들:
# - max_length: 생성할 최대 토큰 수
# - stop_strings: 생성 중단 문자열들
# - repetition_penalty: 반복 방지 페널티 (1.0 이상 권장)
# - temperature: 생성 다양성 조절 (기본값 사용)

output_ids = model.generate(
    **inputs,
    max_length=200,
    stop_strings=["<|endofturn|>", "<|stop|>"],
    repetition_penalty=1.2,
    tokenizer=tokenizer
)

# 결과 출력
generated_text = tokenizer.batch_decode(output_ids)[0]
print("=== 생성된 텍스트 ===")
print(generated_text)

# 원본 입력과 새로 생성된 부분 분리
input_length = inputs['input_ids'].shape[1]
new_tokens = output_ids[0][input_length:]
new_text = tokenizer.decode(new_tokens, skip_special_tokens=True)
print(f"\n=== 새로 생성된 부분만 ===")
print(new_text)

=== 생성된 텍스트 ===
<|im_start|>system
당신은 창의적인 이야기 작가입니다. 주어진 프롬프트로 흥미로운 이야기를 써주세요.<|im_end|>
<|im_start|>user
옛날 옛적에로 시작하는 이야기를 써주세요.<|im_end|>
<|im_start|>assistant
어느 날, 한 마을의 작은 가게에서 일하는 소녀가 있었습니다.

그녀는 매일 아침 일찍 일어나서 가게 문을 열고 손님들을 맞이했습니다. 그녀는 손님의 요구를 듣고 필요한 물건을 찾아주고, 친절하게 안내해주며, 항상 웃으며 일을 했습니다. 그녀의 이름은 '소피아'였습니다.

하루는 소피아가 손님에게 물었습니다.
"안녕하세요! 오늘은 어떤 물건으로 오셨나요?"
손님은 고개를 끄덕이며 대답했습니다.
"저는 오늘 새로운 책을 사려고 합니다."
소피아는 그 말을 듣고 기뻐하며 책장을 열어보았습니다. 그리고 그녀가 찾고 있던 책이 바로 그곳에 있다는 것을 깨달았습니다!
책은 매우 아름답고, 내용이 풍부하고, 그림이 그려져 있어서 정말 마음에 들었습니다!



=== 새로 생성된 부분만 ===
어느 날, 한 마을의 작은 가게에서 일하는 소녀가 있었습니다.

그녀는 매일 아침 일찍 일어나서 가게 문을 열고 손님들을 맞이했습니다. 그녀는 손님의 요구를 듣고 필요한 물건을 찾아주고, 친절하게 안내해주며, 항상 웃으며 일을 했습니다. 그녀의 이름은 '소피아'였습니다.

하루는 소피아가 손님에게 물었습니다.
"안녕하세요! 오늘은 어떤 물건으로 오셨나요?"
손님은 고개를 끄덕이며 대답했습니다.
"저는 오늘 새로운 책을 사려고 합니다."
소피아는 그 말을 듣고 기뻐하며 책장을 열어보았습니다. 그리고 그녀가 찾고 있던 책이 바로 그곳에 있다는 것을 깨달았습니다!
책은 매우 아름답고, 내용이 풍부하고, 그림이 그려져 있어서 정말 마음에 들었습니다!




HyperClova는 자체적으로 autoregressive 모델입니다. "Autoregressive"란, 이전에 생성된 토큰들을 기반으로 다음 토큰을 생성하는 모델을 의미합니다.

위의 코드에서 `model.generate` 메서드는 이미 autoregressive한 방식으로 문장을 생성합니다. 그러나 이를 명시적으로 보여주기 위해 각 단계에서 토큰을 하나씩 생성하는 autoregressive한 코드를 아래에 작성하겠습니다:

## 다양한 한국어 프롬프트로 실험해보기

다음 셀에서 다양한 한국어 프롬프트로 텍스트 생성을 시도해볼 수 있습니다:

In [26]:
# 다양한 한국어 프롬프트로 텍스트 생성 실험
prompts = [
    "인공지능의 미래는",
    "기술이 발달한 세상에서",
    "인생에서 가장 중요한 것은"
]

print("=== 다양한 한국어 프롬프트로 텍스트 생성 실험 ===\n")

for i, prompt in enumerate(prompts, 1):
    print(f"{i}. 프롬프트: '{prompt}'")

    # 채팅 형식으로 구성
    chat = [
        {"role": "system", "content": "당신은 창의적이고 지적인 AI 어시스턴트입니다."},
        {"role": "user", "content": f"{prompt}에 대해 자세히 설명해주세요."}
    ]

    # 템플릿 적용
    inputs = tokenizer.apply_chat_template(
        chat,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    )

    # GPU 사용 가능하면 GPU로 이동
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    # 텍스트 생성
    output_ids = model.generate(
        **inputs,
        max_length=150,
        stop_strings=["<|endofturn|>", "<|stop|>"],
        repetition_penalty=1.2,
        tokenizer=tokenizer
    )

    # 결과 출력
    generated_text = tokenizer.batch_decode(output_ids)[0]
    input_length = inputs['input_ids'].shape[1]
    new_text = tokenizer.decode(output_ids[0][input_length:], skip_special_tokens=True)
    print(f"\n   생성된 텍스트: {new_text}")
    print("-" * 200)

=== 다양한 한국어 프롬프트로 텍스트 생성 실험 ===

1. 프롬프트: '인공지능의 미래는'

   생성된 텍스트: **AI 기술의 발전과 그 영향**

1. **기계 학습 및 딥러닝**: 인공지능은 기계 학습(ML)과 딥 러닝(DL)을 기반으로 합니다. 이러한 기술은 데이터로부터 패턴을 인식하고 예측하는 능력을 향상시켜, 다양한 분야에서 혁신적 변화를 가져오고 있습니다.

2. **자연어 처리(NLP)**: NLP 기술을 통해 컴퓨터가 인간의 언어를 이해하고 생성할 수 있게 되었습니다. 이는 음성 인식부터 번역까지 광범위한 응용 분야로 확장되고 있으며, 의료 진단이나 고객 서비스에도 활용됩니다
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
2. 프롬프트: '기술이 발달한 세상에서'

   생성된 텍스트: **1. 인공지능(AI)의 발전**

인공지능은 컴퓨터가 인간의 지능을 모방하여 학습하고 문제를 해결하는 기술로, 최근 몇 년간 급속도로 발전했습니다.

- **기계학습**: 머신러닝과 딥러닝 기술을 사용하여 데이터를 분석하고 패턴을 인식합니다.
  - 예: 이미지 인식 시스템 (예: 얼굴인식), 자연어 처리(NLP)
  
- **강화학습(Deep Learning)**:
  - 신경망 구조를 기반으로 한 모델로서 복잡한 데이터도 효과적으로 학습할 수 있습니다
--------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [27]:
# HyperClova Autoregressive 생성 과정 분석
# 채팅 템플릿을 사용한 입력 구성
input_text = "옛날 옛적에"
chat = [
    {"role": "system", "content": "당신은 이야기 작가입니다."},
    {"role": "user", "content": f"{input_text}로 시작하는 이야기를 써주세요."}
]

# 템플릿 적용
inputs = tokenizer.apply_chat_template(
    chat,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
)

print(f"원본 프롬프트: {input_text}")
print(f"템플릿 적용 후 토큰 수: {inputs['input_ids'].shape[1]}")
print(f"토큰화된 결과: {inputs['input_ids']}")

if torch.cuda.is_available():
    inputs = {k: v.to("cuda") for k, v in inputs.items()}

원본 프롬프트: 옛날 옛적에
템플릿 적용 후 토큰 수: 31
토큰화된 결과: tensor([[100272,   9125,    198,  65895, 100743, 104848, 101088,  80052,     13,
         100273,    198, 100272,    882,    198,  36092,    249, 104013, 102476,
          82068,  19954,  17835, 101208, 105077, 104144,  92769,     13, 100273,
            198, 100272,  78191,    198]])


In [28]:
# HyperClova Autoregressive한 방식으로 문장 생성
# 각 단계에서 이전 토큰들을 기반으로 다음 토큰을 예측하는 과정을 시각화

# 현재 입력 토큰 수 확인
current_length = inputs['input_ids'].shape[1]
print(f"현재 입력 토큰 수: {current_length}")

max_length = current_length + 30  # 현재 길이에서 20개 토큰 추가 생성
input_ids_concat = inputs['input_ids'].clone()

print("=== HyperClova Autoregressive 생성 과정 ===\n")
print(f"시작 프롬프트: '{input_text}'\n")
print(f"목표 길이: {max_length} 토큰\n")

# 원본 입력 길이 저장 (assistant 응답 부분만 추출하기 위해)
original_length = current_length

step = 0
while input_ids_concat.shape[1] < max_length:
    step += 1

    # 다음 토큰 예측
    model_inputs = {"input_ids": input_ids_concat}
    if "attention_mask" in inputs:
        model_inputs["attention_mask"] = torch.ones_like(input_ids_concat)

    predictions = model(**model_inputs)
    logits = predictions.logits
    predicted_token = torch.argmax(logits[0, -1]).item()

    # 생성된 토큰을 입력 토큰 뒤에 추가 (같은 장치에 맞춤)
    new_token_tensor = torch.tensor([[predicted_token]], device=input_ids_concat.device)
    input_ids_concat = torch.cat([input_ids_concat, new_token_tensor], dim=1)

    # assistant가 생성한 부분만 추출
    assistant_tokens = input_ids_concat[0][original_length:]
    assistant_text = tokenizer.decode(assistant_tokens, skip_special_tokens=True)
    new_token = tokenizer.decode(predicted_token, skip_special_tokens=True)
    print(f"         Assistant 응답: '{assistant_text}")

    # 너무 길어지면 중단
    if step > 30:
        break

print("생성 완료!")

현재 입력 토큰 수: 31
=== HyperClova Autoregressive 생성 과정 ===

시작 프롬프트: '옛날 옛적에'

목표 길이: 61 토큰

         Assistant 응답: '�
         Assistant 응답: '옛
         Assistant 응답: '옛날
         Assistant 응답: '옛날 옛
         Assistant 응답: '옛날 옛적
         Assistant 응답: '옛날 옛적에
         Assistant 응답: '옛날 옛적에,
         Assistant 응답: '옛날 옛적에, 한
         Assistant 응답: '옛날 옛적에, 한 마을
         Assistant 응답: '옛날 옛적에, 한 마을에
         Assistant 응답: '옛날 옛적에, 한 마을에 한
         Assistant 응답: '옛날 옛적에, 한 마을에 한 소녀
         Assistant 응답: '옛날 옛적에, 한 마을에 한 소녀가
         Assistant 응답: '옛날 옛적에, 한 마을에 한 소녀가 살고
         Assistant 응답: '옛날 옛적에, 한 마을에 한 소녀가 살고 있
         Assistant 응답: '옛날 옛적에, 한 마을에 한 소녀가 살고 있�
         Assistant 응답: '옛날 옛적에, 한 마을에 한 소녀가 살고 있었습니다
         Assistant 응답: '옛날 옛적에, 한 마을에 한 소녀가 살고 있었습니다.
         Assistant 응답: '옛날 옛적에, 한 마을에 한 소녀가 살고 있었습니다. 그녀의
         Assistant 응답: '옛날 옛적에, 한 마을에 한 소녀가 살고 있었습니다. 그녀의 이름은
         Assistant 응답: '옛날 옛적에, 한 마을에 한 소녀가 살고 있었습니다. 그녀의 이름은 '
         Assistant 응답: '옛날 옛적에, 한 마을에